<a href="https://colab.research.google.com/github/custerc/miscellaneous-marketing-code/blob/main/Blog_Traffic_Drop_Analysis_for_Cockroach_Labs_V2_(GA4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd

#create empty lists for data from the current and previous periods in the export
current_list = []
previous_list = []

#open the csv file
with open (str('data-export.csv'), 'r') as file:
  data_reader=csv.reader(file)
  datalist = list(data_reader)

# Flag to determine whether to add rows to 'current' or 'previous'
add_to_current = True

for row in datalist[9:]:
# handle the empty rows and second round of dates
  if not row:
    continue
  if 'Start date' in row[0]:
    continue
  if 'End date' in row[0]:
    continue

  # keep add_to_current true until you hit # All Users
  if row[0] == '# All Users':
    add_to_current = False
    continue  # Skip the row containing '# All Users'

  # appending to the appropriate lists
  if add_to_current:
    current_list.append(row)
  else:
    previous_list.append(row)


Current:
[['Landing page', 'Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue'], ['/blog/what-is-a-foreign-key', '8361', '7718', '7480', '52.84642985288841', '11', '0'], ['/blog/sql-add-constraint', '6810', '6376', '6101', '41.09500734214391', '3', '0'], ['/blog/what-is-a-uuid', '5791', '5264', '5072', '45.86997064410292', '11', '0'], ['/blog/select-for-update', '5456', '4879', '4529', '41.7025293255132', '0', '0']]

Previous:
[['Landing page', 'Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue'], ['/blog/what-is-a-foreign-key', '6262', '5611', '5449', '54.9366017246886', '0', '0'], ['/blog/sql-add-constraint', '5776', '5285', '5089', '39.21537396121884', '0', '0'], ['/blog/what-is-a-uuid', '5250', '4836', '4709', '48.276761904761905', '11', '0'], ['/blog/select-for-update', '5458', '4750', '4418', '45.110113594723344', '3', '0']]


In [34]:
# create two dataframes with the two lists
df_current = pd.DataFrame(current_list,columns=['Landing page', 'Sessions', 'Users', 'New users', 'Average engagement time per session', 'Conversions', 'Total revenue'])
df_previous = pd.DataFrame(previous_list,columns=['Landing page', 'Sessions_prev', 'Users_prev', 'New users_prev', 'Average engagement time per session_prev', 'Conversions_prev', 'Total revenue_prev'])

# merge the dataframes and clean the data
joined = pd.merge(df_current, df_previous, on='Landing page', how='outer')
joined.replace(',','', regex=True, inplace=True)
joined.replace('%','', regex=True, inplace=True)
joined = joined.drop(index=0)


# convert to numeric datatypes
df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)




joined.dtypes


Landing page                                 object
Sessions                                     object
Users                                        object
New users                                    object
Average engagement time per session          object
Conversions                                  object
Total revenue                                object
Sessions_prev                                 int64
Users_prev                                   object
New users_prev                               object
Average engagement time per session_prev     object
Conversions_prev                             object
Total revenue_prev                           object
SessionsDelta_perc                          float64
SessionsDelta_raw                           float64
dtype: object

In [ ]:

# create new columns tracking the percentage and raw change in sessions in the joined df


joined['SessionsDelta_perc'] = (joined['Sessions'].astype('float') - joined['Sessions_prev'].astype('float')) / joined['Sessions_prev'].astype('float') * 100
joined['SessionsDelta_raw'] = (joined['Sessions'].astype('float') - joined['Sessions_prev'].astype('float'))
joined = joined.round({'SessionsDelta_perc': 1})
joined['Sessions_prev'] = pd.to_numeric(joined['Sessions_prev'])



# create the report to output - WORK IN PROGRESS

output = joined.sort_values(by=['SessionsDelta_perc'], ascending=True)
# output = output.drop(output[output.Sessions_prev < 50].index, inplace=True)
output = output[['Landing page', 'SessionsDelta_perc', 'SessionsDelta_raw', 'Average engagement time per session', 'Average engagement time per session_prev']]

output.head(500)